<a id="data_set"></a>
# 1: Segmentation of WAVs

The script reads each WAV file, finding periods of silence or noise, and segments rather than manually annotating the files.
- Birdsong recordings are obtained by recording through the smart aviary (? <-- confirm this) chiefly developed by the Schmidt Lab at the University of Pennsylvania.


## Imports

In [123]:
import numpy as np
import sys
import os
from tqdm import tqdm_notebook as tqdm
from glob import glob
import re 
from datetime import datetime, timedelta
import pandas as pd 
import os
from sklearn.externals.joblib import Parallel, delayed
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
# imports of local methods from the source code
from avgn.segment_song.preprocessing import *
import avgn.segment_song.preprocessing as pp

In [118]:
# output location of hdf5 files
output_location = '../data/cb_wavs'

## Find and Analyze WAVs
- Relative to each bird.
- Input location found in ../data/cb_inputs/.
- Output as an hdf5 file in cb_wavs.
- Prints out dset structure.

In [119]:
# must save recordings to ../data/cb_inputs
# recordings can be found in data file in avgn
input_loc = '../data/cb_inputs'

In [120]:
# input and specify file types when recordings are received'
dsets = [(input_loc, 'cb' )]

In [121]:
print(dsets)

[('../data/cb_inputs', 'cb')]


In [122]:
wav_list = np.array([])
dset_list = np.array([])
for search_directory, dset in tqdm(dsets):
    new_wavs = np.array(glob(search_directory))
    dset_list = np.append(dset_list, [dset for i in range(len(new_wavs))])
    print(dset_list)
    wav_list = np.append(wav_list,new_wavs)
    print(wav_list)
print(wav_list[0], len(wav_list))

[]
[]



IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# organizes birds by name - unnamed waves (irrelvant?)
# bird_names = [i.split('/')[6] for i in wav_list]
# print(np.unique(bird_names))

## Create WAV Dataframe
- Extract bird times and create the dataframe of the wavs. 

In [ ]:
import xml.etree.ElementTree

In [ ]:
bir_xml_locs = glob(input_loc+'/*/Annotation.xml')
bird_xml_locs[:2]

### Custom parsing of an XML file to get wav time information.

In [ ]:
song_df = pd.DataFrame(columns=['bird','WavLoc', 'WaveFileName','Position','Length', 'NumNote', 'NotePositions', 'NoteLengths', 'NoteLabels'])

for bird_loc in tqdm(bird_xml_locs):
    bird_xml = xml.etree.ElementTree.parse(bird_loc).getroot()
    bird = bird_loc.split('/')[-2]
    for element in tqdm(bird_xml.getchildren(), leave=False):
        if element.tag == 'Sequence':
            notePositions = []
            noteLengths = []
            noteLabels = []
            for seq_element in element.getchildren():
                if seq_element.tag == 'Position': position = seq_element.text
                elif seq_element.tag == 'Length': length = seq_element.text
                elif seq_element.tag == 'WaveFileName': WaveFileName = seq_element.text
                elif seq_element.tag == 'NumNote': NumNote = seq_element.text
                elif seq_element.tag == 'Note':
                    for note_element in seq_element.getchildren():
                        if note_element.tag == 'Label': noteLabels.append(note_element.text)
                        elif note_element.tag == 'Position': notePositions.append(note_element.text)
                        elif note_element.tag == 'Length': noteLengths.append(note_element.text)
            song_df.loc[len(song_df)] = [bird, input_loc+bird+'/Wave/'+WaveFileName, WaveFileName, position, length, NumNote, notePositions, noteLengths, noteLabels]

In [ ]:
# default date time method
wav_times = []
wav_loc = wav_list[0]
n_no_date = 0
for wav_file in wav_list:
        dt = datetime(1900, 1, 1, 0, 0) + timedelta(hours=n_no_date)
        n_no_date+=1
        wav_times.append(dt)    
wav_times = np.array(wav_times)

In [ ]:
wav_times[:3]

In [ ]:
# pandas dataframe corresponding to files, datetimes
wav_df = pd.DataFrame.from_dict({'filename':wav_list,
                                'wav_time': wav_times,
                                'dset': dset_list,
                                'birdname': bird_names})
wav_df[:3]

## Processing Vocalizations
- Specific to brown-headed cowbirds.
- **TODO: SET CUSTOM PARAMETERS!!**

In [ ]:
param_dict = {}
param_dict['cb'] = {
    ### Parameters ###
    'lowcut': 0, # Hz # Low cut for our butter bandpass filter
    'highcut': 15000, # Hz # High cut for our butter bandpass filter

    'rms_window':  5, # seconds # the size of your window
    'rms_stride': .01, # seconds # how big your step size should be for moving the filter
    'noise_thresh': .01, # threshold percent of maximum noise to consider silence
    'segment_padding': 4.0, # seconds to pad waveform extracted
    'rms_padding': 1.0, #5.0, # seconds # how much to pad around vocalizations
   
    # filtering
    'min_amp_val': 0, # the minimum value of a wav's amplitude to be considered containing any sound
    'min_segment_length_s': 0, # How long a bout has to be to count
    'max_segment_length_s': 10000.,  # If a bout is too long, dont count it
    'min_silence_pct': 0.05,  # measure of noise in wav, by threshing the pct of time that the wav is silent

    # FFT (we create a spectrogram here to filter out noise)
    'num_freq':1024, # how many channels to use in a spectrogram 
    'sample_rate':44100, # what rate are your WAVs sampled at?
    'preemphasis':0.97, 
    'ref_level_db':20, # reference db for computing spec
    'frame_shift_ms':2, # step size for fft
    'min_level_db': -50,# threshold for spectrograms (lower filters out more noise)
    'max_power_f_min': 1000,# (HZ) If the maximum power of the spectral envelope is below this, call noise
    'frame_shift_ms':40, # step size for fft
    'frame_length_ms':40, # frame length for fft

    # # Filter based upon power-frequency envelope
    'vocal_freq_min' : 700,
    'vocal_freq_max' : 15000
    }

In [ ]:
# print the number of wav files found for each individual
for (dset, bird), group in wav_df.groupby(('dset', 'birdname')):
    print(dset, bird, len(group))

## Preprocessing and Debugging

In [ ]:
skip_created = True # whether to skip song that has already been processed
parallel = False # whether to run this algorithm in parallel (across wav files)
visualize = False # whether to output visualizations of spectrograms to the notebook screen - this is useful for setting parameters - you may also want to edit the code to visualized other aspects of the algorithm
n_parallel = 10 # How many threads to run in parallel (if parallel == True)
verbosity = 1 # how verbose to make the output of the parallelization (higher = more, 0 = none, >50 output is sent to std.out)
verbose=False

### Saving WAVs
- Saves to ''../data/cb_waves'.
- Saves spectrogram PNGs to given folder to view segmentation algorithm.

In [ ]:
save_to_folder = '../data/cb_wavs' # Where to save output wavs
save_spectrograms = True 

In [ ]:
from soundsig.sound import BioSound 
from soundsig.sound import WavFile

In [ ]:
# loop through all wavs, 
# TODO: find way to group by birdname
try:
    key_list = ('wav_list', 'time_index', 'wav_file', 'wav_time', 'rate')
    for (dset, bird), group in wav_df.groupby(('dset', 'birdname')):   

        print('processing %s to save at %s' % (bird, save_to_folder))
        bird_data = {key : [] for key in key_list}

        print('total wavs: ', len(group)) 

        # create a spot to save the data
        bird_folder = save_to_folder+bird+'/'
        if not os.path.exists(bird_folder+'wavs/'):
            os.makedirs(bird_folder+'wavs/') 
        if not os.path.exists(bird_folder+'csv/'):
            os.makedirs(bird_folder+'csv/') 

        if parallel:
            with Parallel(n_jobs=n_parallel, verbose=verbosity) as parallel:
                parallel(delayed(pp.process_bird_wav)(bird, filename, wav_time, param_dict[dset],save_to_folder,
                                                      visualize= visualize, skip_created= skip_created,
                                                      save_spectrograms= save_spectrograms, verbose=verbose) 
                                                      for idx, gbird, gdset, filename, wav_time in tqdm(group.itertuples(),total=len(group)))
        else:
            for idx, gbird, gdset, filename, wav_time in tqdm(group.itertuples(), total=len(group)):
                process_bird_wav(bird, filename, wav_time, param_dict[dset],save_to_folder, visualize=visualize,
                                 skip_created=skip_created, save_spectrograms= save_spectrograms, verbose=verbose) 
except KeyboardInterrupt:
    print('interrrupted')